In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

 50%|████▉     | 17874/36000 [00:00<00:00, 21863.44it/s]

Error parsing /home/dzc/code/work/goose/experiments/_log_plan/pfcalcul/miconic_wl_collapse-all_none_rank-svm_plan_fd_2_2_14_0.log: 'NoneType' object has no attribute 'group'


100%|██████████| 36000/36000 [00:01<00:00, 30446.74it/s]


In [3]:
COLUMNS = [
    "data_pruning",
    "facts",
    "optimiser",
    "features",
    "iterations",
    "feature_pruning",
]

def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "fd"]
    df = df[df.features == "wl"]
    df = df[df.data_pruning == "equivalent-weighted"]
    # df = df[df.feature_pruning.isin({"none", "collapse-all"})]
    return df

for metric in [
    "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    # df = df.fillna(0)
    # replace 0 with NaN
    # df = df.replace(0, -1)
    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)

tried


domain                                                                    blocksworld  \
data_pruning        facts optimiser features iterations feature_pruning                 
equivalent-weighted fd    rank-svm  wl       2          collapse-all               87   
                                                        collapse-all-x              0   
                                                        collapse-layer             89   
                                                        collapse-layer-x           88   
                                                        none                       90   
                                             4          collapse-all               90   
                                                        collapse-all-x              0   
                                                        collapse-layer             89   
                                                        collapse-layer-x           89   
                                                        none                       90   
                          svr       wl       2          collapse-all               90   
                                                        collapse-all-x              0   
                                                        collapse-layer             90   
                                                        collapse-layer-x           90   
                                                        none                       90   
                                             4          collapse-all               90   
                                                        collapse-all-x              0   
                                                        collapse-layer             90   
                                                        collapse-layer-x           90   
                                                        none                       90   

domain                                                                    childsnack  \
data_pruning        facts optimiser features iterations feature_pruning                
equivalent-weighted fd    rank-svm  wl       2          collapse-all              90   
                                                        collapse-all-x             0   
                                                        collapse-layer            90   
                                                        collapse-layer-x          90   
                                                        none                      90   
                                             4          collapse-all              89   
                                                        collapse-all-x             0   
                                                        collapse-layer            89   
                                                        collapse-layer-x          90   
                                                        none                      90   
                          svr       wl       2          collapse-all              90   
                                                        collapse-all-x             0   
                                                        collapse-layer            90   
                                                        collapse-layer-x          90   
                                                        none                      90   
                                             4          collapse-all              90   
                                                        collapse-all-x             0   
                                                        collapse-layer            90   
                                                        collapse-layer-x          90   
                                                        none                      89   

domain                                                                    ferry  \
data_pruning        f

solved


domain                                                                    blocksworld  \
data_pruning        facts optimiser features iterations feature_pruning                 
equivalent-weighted fd    rank-svm  wl       2          collapse-all               70   
                                                        collapse-all-x              0   
                                                        collapse-layer             67   
                                                        collapse-layer-x           47   
                                                        none                       73   
                                             4          collapse-all               71   
                                                        collapse-all-x              0   
                                                        collapse-layer             65   
                                                        collapse-layer-x           47   
                                                        none                       69   
                          svr       wl       2          collapse-all               35   
                                                        collapse-all-x              0   
                                                        collapse-layer             33   
                                                        collapse-layer-x           48   
                                                        none                       32   
                                             4          collapse-all               55   
                                                        collapse-all-x              0   
                                                        collapse-layer             32   
                                                        collapse-layer-x           46   
                                                        none                       61   

domain                                                                    childsnack  \
data_pruning        facts optimiser features iterations feature_pruning                
equivalent-weighted fd    rank-svm  wl       2          collapse-all              26   
                                                        collapse-all-x             0   
                                                        collapse-layer            28   
                                                        collapse-layer-x          31   
                                                        none                      25   
                                             4          collapse-all              33   
                                                        collapse-all-x             0   
                                                        collapse-layer            38   
                                                        collapse-layer-x          31   
                                                        none                      32   
                          svr       wl       2          collapse-all              18   
                                                        collapse-all-x             0   
                                                        collapse-layer            18   
                                                        collapse-layer-x          22   
                                                        none                      18   
                                             4          collapse-all              18   
                                                        collapse-all-x             0   
                                                        collapse-layer            20   
                                                        collapse-layer-x          22   
                                                        none                      31   

domain                                                                    ferry  \
data_pruning        f

In [4]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        # print(len(df))
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"] + " - " + df["data_pruning"] # + " - " + df["feature_pruning"]
        # df["config"] = " _ ".join([df[f] for f in ["features", "iterations", "optimiser", "data_pruning", "feature_pruning"]])
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="feature_pruning",
            line_dash="feature_pruning",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        fig.update_xaxes(categoryorder='array', categoryarray=PROBLEMS)
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)